### Docs

https://github.com/aws/amazon-sagemaker-examples/blob/main/serverless-inference/serverless-model-registry.ipynb

In [1]:
# Libraries importation
import pandas as pd
import numpy as np
import random
import os
import time
from sklearn.model_selection import train_test_split
import boto3
import sagemaker
from sagemaker.estimator import Estimator

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
# Lista de exemplos de navegadores da web
web_browsers = ['Chrome', 'Firefox', 'Safari', 'Edge', 'Opera']

# Lista de exemplos de dispositivos
devices = ['Desktop', 'Laptop', 'Smartphone', 'Tablet']

# Lista de exemplos de sistemas operacionais
operating_systems = ['Windows', 'macOS', 'Linux', 'iOS', 'Android']

# Lista de categorias de tempo para previsão
weather_categories = ['SUNNY', 'CLOUDY', 'RAINY', 'FOG', 'SNOW']

# Lista de coordenadas de latitude e longitude de exemplo
coordinates = [
    {'latitude': -23.5505, 'longitude': -46.6333},  # Exemplo de São Paulo, Brasil
    {'latitude': 40.7128, 'longitude': -74.0060},  # Exemplo de Nova York, EUA
    {'latitude': 51.5074, 'longitude': -0.1278},   # Exemplo de Londres, Reino Unido
    {'latitude': 35.6895, 'longitude': 139.6917},  # Exemplo de Tóquio, Japão
    {'latitude': -33.8688, 'longitude': 151.2093}  # Exemplo de Sydney, Austrália
]

# Função para gerar um exemplo de dados aleatório
def generate_example():
    example = {}
    # Seleciona aleatoriamente uma categoria de tempo para previsão
    example['PREDICT_WEATHER_CATEGORY'] = random.choice(weather_categories)
    # Seleciona aleatoriamente uma coordenada de latitude e longitude
    coordinates_choice = random.choice(coordinates)
    example['LATITUDE'] = coordinates_choice['latitude']
    example['LONGITUDE'] = coordinates_choice['longitude']
    # Gera um timestamp aleatório (apenas para ilustração)
    example['TIMESTAMP'] = '2024-02-24 12:00:00'
    # Seleciona aleatoriamente um navegador, dispositivo e sistema operacional
    example['WEB_BROWSER'] = random.choice(web_browsers)
    example['DEVICE'] = random.choice(devices)
    example['OPERATING_SYSTEM'] = random.choice(operating_systems)
    return example

# Gera 100 exemplos de dados
dataset = [generate_example() for _ in range(10000)]

index_to_class = {0: 'CLOUDY', 1: 'FOG', 2: 'RAINY', 3: 'SNOW', 4: 'SUNNY'}
CLASS_INDEX = {'CLOUDY': 0, 'FOG': 1, 'RAINY': 2, 'SNOW': 3, 'SUNNY': 4}

dataset_df = pd.DataFrame(dataset)
dataset_df['PREDICT_WEATHER_CATEGORY'] = dataset_df['PREDICT_WEATHER_CATEGORY'].map(CLASS_INDEX)
dataset_df

,PREDICT_WEATHER_CATEGORY,LATITUDE,LONGITUDE,TIMESTAMP,WEB_BROWSER,DEVICE,OPERATING_SYSTEM
0,4,35.6895,139.6917,2024-02-24 12:00:00,Chrome,Smartphone,Linux
1,1,51.5074,-0.1278,2024-02-24 12:00:00,Edge,Desktop,Linux
2,1,40.7128,-74.0060,2024-02-24 12:00:00,Opera,Desktop,Android
3,1,-23.5505,-46.6333,2024-02-24 12:00:00,Chrome,Laptop,Linux
4,2,51.5074,-0.1278,2024-02-24 12:00:00,Firefox,Desktop,Android
...,...,...,...,...,...,...,...
9995,4,-33.8688,151.2093,2024-02-24 12:00:00,Safari,Laptop,macOS
9996,2,35.6895,139.6917,2024-02-24 12:00:00,Opera,Laptop,Linux
9997,3,35.6895,139.6917,2024-02-24 12:00:00,Safari,Tablet,Android
9998,0,35.6895,139.6917,2024-02-24 12:00:00,Chrome,Desktop,macOS


# Feature Engineering

In [3]:
def feature_engineering(dataset_df):
    WEB_BROWSER = {'Chrome': 0, 'Edge': 1, 'Firefox':3, 'Opera':4, 'Safari': 5}
    DEVICE = {'Desktop':0, 'Laptop':1, 'Smartphone':2, 'Tablet':3}
    OPERATING_SYSTEM = {'Android':0, 'Linux':1, 'Windows':2, 'iOS':3, 'macOS':4}

    dataset_df['WEB_BROWSER'] = dataset_df['WEB_BROWSER'].map(WEB_BROWSER)
    dataset_df['DEVICE'] = dataset_df['DEVICE'].map(DEVICE)
    dataset_df['OPERATING_SYSTEM'] = dataset_df['OPERATING_SYSTEM'].map(OPERATING_SYSTEM)
    
    return dataset_df
    
dataset_fe = feature_engineering(dataset_df)
dataset_fe

,PREDICT_WEATHER_CATEGORY,LATITUDE,LONGITUDE,TIMESTAMP,WEB_BROWSER,DEVICE,OPERATING_SYSTEM
0,4,35.6895,139.6917,2024-02-24 12:00:00,0,2,1
1,1,51.5074,-0.1278,2024-02-24 12:00:00,1,0,1
2,1,40.7128,-74.0060,2024-02-24 12:00:00,4,0,0
3,1,-23.5505,-46.6333,2024-02-24 12:00:00,0,1,1
4,2,51.5074,-0.1278,2024-02-24 12:00:00,3,0,0
...,...,...,...,...,...,...,...
9995,4,-33.8688,151.2093,2024-02-24 12:00:00,5,1,4
9996,2,35.6895,139.6917,2024-02-24 12:00:00,4,1,1
9997,3,35.6895,139.6917,2024-02-24 12:00:00,5,3,0
9998,0,35.6895,139.6917,2024-02-24 12:00:00,0,0,4


In [4]:
# Assuming you already have a dataset where 'target' is your target variable
# and 'features' are your input features.

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset_fe.drop(columns=['PREDICT_WEATHER_CATEGORY']), 
                                                    dataset_fe['PREDICT_WEATHER_CATEGORY'], 
                                                    test_size=0.2, 
                                                    random_state=42)


# Concatenar X_train e y_train em um único DataFrame
train_data = pd.concat([y_train, X_train], axis=1)

# Concatenar X_test e y_test em um único DataFrame
test_data = pd.concat([y_test, X_test], axis=1)

# Salvar os conjuntos de dados de treinamento e teste juntos em um único arquivo CSV
train_data.to_csv('train.csv', index=False)
test_data.to_csv('test.csv', index=False)

In [5]:
# Bot client
s3 = boto3.client('s3')

# Diretório onde os arquivos CSV estão localizados
directory = f"{os.getcwd()}/"

# Conecte-se ao cliente S3
s3 = boto3.client('s3')

# Upload do arquivo train.csv
train_file = os.path.join(directory, 'train.csv')
s3.upload_file(train_file, 'weatherwizard-ml-model', 'data/train.csv')

# Upload do arquivo test.csv
test_file = os.path.join(directory, 'test.csv')
s3.upload_file(test_file, 'weatherwizard-ml-model', 'data/test.csv')

# Setup

In [6]:
# Setup clients
client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")

In [7]:
boto_session = boto3.session.Session()
region = boto_session.region_name
print(region)

sagemaker_session = sagemaker.Session()
base_job_prefix = "xgboost-weatherwizard-ml-model"
role = sagemaker.get_execution_role()
print(role)

#default_bucket = f'weatherwizard-ml-model-{sagemaker_session.default_bucket()}'
default_bucket = 'weatherwizard-ml-model'
s3_prefix = base_job_prefix

training_instance_type = "ml.m5.xlarge"

us-east-1
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
arn:aws:iam::279288473542:role/service-role/AmazonSageMaker-ExecutionRole-20240313T183292


# Model Training

In [8]:
from sagemaker.inputs import TrainingInput

training_path = f"s3://{default_bucket}/data/train.csv"
train_input = TrainingInput(training_path, content_type="text/csv")

In [9]:
model_path = f"s3://{default_bucket}/{s3_prefix}/xgb_model"

# retrieve xgboost image
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type=training_instance_type,
)

# Configure Training Estimator
xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=model_path,
    sagemaker_session=sagemaker_session,
    role=role,
)

# Set Hyperparameters
xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0,
)

In [10]:
# Fit model
xgb_train.fit({"train": train_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-05-18-18-46-05-621


2024-05-18 18:46:05 Starting - Starting the training job...
2024-05-18 18:46:21 Starting - Preparing the instances for training...
2024-05-18 18:46:47 Downloading - Downloading input data...
2024-05-18 18:47:12 Downloading - Downloading the training image...
2024-05-18 18:47:58 Training - Training image download completed. Training in progress.
2024-05-18 18:47:58 Uploading - Uploading generated training model[2024-05-18 18:47:54.011 ip-10-0-253-181.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Sing

In [11]:
# Retrieve model data from training job
model_artifacts = xgb_train.model_data
model_artifacts

's3://weatherwizard-ml-model/xgboost-weatherwizard-ml-model/xgb_model/sagemaker-xgboost-2024-05-18-18-46-05-621/output/model.tar.gz'

# Model Registry

In [12]:
# Create a Model Package Group: https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-model-group.html
import time
from time import gmtime, strftime

model_package_group_name = "xgboost-weather-wizard" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_package_group_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageGroupDescription": "Model package group for xgboost regression model",
}

create_model_pacakge_group_response = client.create_model_package_group(
    **model_package_group_input_dict
)
print(
    "ModelPackageGroup Arn : {}".format(create_model_pacakge_group_response["ModelPackageGroupArn"])
)

ModelPackageGroup Arn : arn:aws:sagemaker:us-east-1:279288473542:model-package-group/xgboost-weather-wizard2024-05-18-18-48-48


In [13]:
model_package_group_arn = create_model_pacakge_group_response["ModelPackageGroupArn"]
modelpackage_inference_specification = {
    "InferenceSpecification": {
        "Containers": [
            {
                "Image": image_uri,
            }
        ],
        "SupportedContentTypes": ["text/csv"],
        "SupportedResponseMIMETypes": ["text/csv"],
    }
}

# Specify the model source
model_url = model_artifacts

# Specify the model data
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0][
    "ModelDataUrl"
] = model_url

create_model_package_input_dict = {
    "ModelPackageGroupName": model_package_group_arn,
    "ModelPackageDescription": "Model for regression Weather Wizard dataset",
    "ModelApprovalStatus": "PendingManualApproval",
}
create_model_package_input_dict.update(modelpackage_inference_specification)

# Create cross-account model package
create_mode_package_response = client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print("ModelPackage Version ARN : {}".format(model_package_arn))

ModelPackage Version ARN : arn:aws:sagemaker:us-east-1:279288473542:model-package/xgboost-weather-wizard2024-05-18-18-48-48/1


In [14]:
client.list_model_packages(ModelPackageGroupName=model_package_group_name)

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'xgboost-weather-wizard2024-05-18-18-48-48',
   'ModelPackageVersion': 1,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:279288473542:model-package/xgboost-weather-wizard2024-05-18-18-48-48/1',
   'ModelPackageDescription': 'Model for regression Weather Wizard dataset',
   'CreationTime': datetime.datetime(2024, 5, 18, 18, 48, 48, 543000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'}],
 'ResponseMetadata': {'RequestId': 'eb15603f-31a7-4122-9c6c-a6efb8d64481',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'eb15603f-31a7-4122-9c6c-a6efb8d64481',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '425',
   'date': 'Sat, 18 May 2024 18:48:48 GMT'},
  'RetryAttempts': 0}}

In [15]:
model_package_arn = client.list_model_packages(ModelPackageGroupName=model_package_group_name)[
    "ModelPackageSummaryList"
][0]["ModelPackageArn"]
model_package_arn

'arn:aws:sagemaker:us-east-1:279288473542:model-package/xgboost-weather-wizard2024-05-18-18-48-48/1'

In [16]:
client.describe_model_package(ModelPackageName=model_package_arn)

{'ModelPackageGroupName': 'xgboost-weather-wizard2024-05-18-18-48-48',
 'ModelPackageVersion': 1,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:279288473542:model-package/xgboost-weather-wizard2024-05-18-18-48-48/1',
 'ModelPackageDescription': 'Model for regression Weather Wizard dataset',
 'CreationTime': datetime.datetime(2024, 5, 18, 18, 48, 48, 543000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3',
    'ImageDigest': 'sha256:b517ba0196d2579c5393750335ed104da74b11ad2a6f0ae5da6907a7221ff2e0',
    'ModelDataUrl': 's3://weatherwizard-ml-model/xgboost-weatherwizard-ml-model/xgb_model/sagemaker-xgboost-2024-05-18-18-46-05-621/output/model.tar.gz'}],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']},
 'ModelPackageStatus': 'Completed',
 'ModelPackageStatusDetails': {'ValidationStatuses': [],
  'ImageScanStatuses': []},
 'CertifyForMarketplace': 

In [17]:
# Approve the model package
model_package_update_input_dict = {
    "ModelPackageArn": model_package_arn,
    "ModelApprovalStatus": "Approved",
}
model_package_update_response = client.update_model_package(**model_package_update_input_dict)
print(model_package_update_response)

{'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:279288473542:model-package/xgboost-weather-wizard2024-05-18-18-48-48/1', 'ResponseMetadata': {'RequestId': 'f23571db-4e68-4376-aa8e-2de95c16fef3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f23571db-4e68-4376-aa8e-2de95c16fef3', 'content-type': 'application/x-amz-json-1.1', 'content-length': '120', 'date': 'Sat, 18 May 2024 18:48:48 GMT'}, 'RetryAttempts': 0}}


# Deployment

## Model Creation

In [18]:
model_name = "xgboost-weather-wizard-serverless-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name : {}".format(model_name))
container_list = [{"ModelPackageName": model_package_arn}]

create_model_response = client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, Containers=container_list
)
print("Model arn : {}".format(create_model_response["ModelArn"]))

Model name : xgboost-weather-wizard-serverless-model2024-05-18-18-48-48
Model arn : arn:aws:sagemaker:us-east-1:279288473542:model/xgboost-weather-wizard-serverless-model2024-05-18-18-48-48


## Endpoint Configuration Creation

This is where you can adjust the Serverless Configuration for your endpoint. The current max concurrent invocations for a single endpoint, known as MaxConcurrency, can be any value from 1 to 200, and MemorySize can be any of the following: 1024 MB, 2048 MB, 3072 MB, 4096 MB, 5120 MB, or 6144 MB.

In [19]:
try:
    endpoint_config_name = "xgboost-serverless-weather-wizard-endpoint-configuration"# + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    print(endpoint_config_name)
    create_endpoint_config_response = client.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[
            {
                "ServerlessConfig": {"MemorySizeInMB": 1024, "MaxConcurrency": 10},
                "ModelName": model_name,
                "VariantName": "AllTraffic",
            }
        ],
    )
    print("Endpoint Configuration Arn: " + create_endpoint_config_response["EndpointConfigArn"])
except:
    print("Endpoint configuration already created")

xgboost-serverless-weather-wizard-endpoint-configuration
Endpoint configuration already created


In [20]:
endpoint_config_name = "xgboost-serverless-weather-wizard-endpoint-configuration"

def create_or_update_endpoint_config(client, endpoint_config_name, model_name):
    try:
        # Verifique se o endpoint configuration já existe
        existing_configs = client.list_endpoint_configs(NameContains=endpoint_config_name)
        for config in existing_configs['EndpointConfigs']:
            if config['EndpointConfigName'] == endpoint_config_name:
                print(f"Endpoint configuration '{endpoint_config_name}' already exists. Updating it.")

                # Delete o endpoint configuration existente
                client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
                print(f"Deleted existing endpoint configuration '{endpoint_config_name}'.")
                break

        # Crie o novo endpoint configuration
        create_endpoint_config_response = client.create_endpoint_config(
            EndpointConfigName=endpoint_config_name,
            ProductionVariants=[
                {
                    "ServerlessConfig": {"MemorySizeInMB": 1024, "MaxConcurrency": 10},
                    "ModelName": model_name,
                    "VariantName": "AllTraffic",
                }
            ],
        )
        print("Endpoint Configuration Arn: " + create_endpoint_config_response["EndpointConfigArn"])
    except client.exceptions.ResourceNotFound:
        # Se o endpoint configuration não existe, crie um novo
        print(f"Endpoint configuration '{endpoint_config_name}' does not exist. Creating it.")
        create_endpoint_config_response = client.create_endpoint_config(
            EndpointConfigName=endpoint_config_name,
            ProductionVariants=[
                {
                    "ServerlessConfig": {"MemorySizeInMB": 1024, "MaxConcurrency": 10},
                    "ModelName": model_name,
                    "VariantName": "AllTraffic",
                }
            ],
        )
        print("Endpoint Configuration Arn: " + create_endpoint_config_response["EndpointConfigArn"])
    except Exception as e:
        print(f"An error occurred: {e}")

create_or_update_endpoint_config(client, endpoint_config_name, model_name)


Endpoint configuration 'xgboost-serverless-weather-wizard-endpoint-configuration' already exists. Updating it.
Deleted existing endpoint configuration 'xgboost-serverless-weather-wizard-endpoint-configuration'.
Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:279288473542:endpoint-config/xgboost-serverless-weather-wizard-endpoint-configuration


## Endpoint Creation

In [21]:
endpoint_name = "xgboost-weather-wizard-serverless-endpoint"

def create_or_update_endpoint(client, endpoint_name, endpoint_config_name):
    try:
        print(f"Checking if the endpoint '{endpoint_name}' already exists.")
        existing_endpoints = client.list_endpoints(NameContains=endpoint_name)
        
        for endpoint in existing_endpoints['Endpoints']:
            if endpoint['EndpointName'] == endpoint_name:
                print(f"Endpoint '{endpoint_name}' already exists. Preparing to update it.")

                print(f"Deleting the existing endpoint '{endpoint_name}'.")
                client.delete_endpoint(EndpointName=endpoint_name)

                print(f"Waiting for the endpoint '{endpoint_name}' to be completely deleted.")
                waiter = client.get_waiter('endpoint_deleted')
                waiter.wait(EndpointName=endpoint_name)
                print(f"Endpoint '{endpoint_name}' has been successfully deleted.")
                break

        print(f"Creating a new endpoint with name '{endpoint_name}'.")
        create_endpoint_response = client.create_endpoint(
            EndpointName=endpoint_name,
            EndpointConfigName=endpoint_config_name
        )
        print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])
    
    except client.exceptions.ResourceNotFound:
        print(f"Endpoint '{endpoint_name}' does not exist. Creating it.")
        create_endpoint_response = client.create_endpoint(
            EndpointName=endpoint_name,
            EndpointConfigName=endpoint_config_name
        )
        print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])
    
    except Exception as e:
        print(f"An error occurred: {e}")

create_or_update_endpoint(client, endpoint_name, endpoint_config_name)


Checking if the endpoint 'xgboost-weather-wizard-serverless-endpoint' already exists.
Endpoint 'xgboost-weather-wizard-serverless-endpoint' already exists. Preparing to update it.
Deleting the existing endpoint 'xgboost-weather-wizard-serverless-endpoint'.
Waiting for the endpoint 'xgboost-weather-wizard-serverless-endpoint' to be completely deleted.
Endpoint 'xgboost-weather-wizard-serverless-endpoint' has been successfully deleted.
Creating a new endpoint with name 'xgboost-weather-wizard-serverless-endpoint'.
Endpoint Arn: arn:aws:sagemaker:us-east-1:279288473542:endpoint/xgboost-weather-wizard-serverless-endpoint


In [22]:
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService


{'EndpointName': 'xgboost-weather-wizard-serverless-endpoint',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:279288473542:endpoint/xgboost-weather-wizard-serverless-endpoint',
 'EndpointConfigName': 'xgboost-serverless-weather-wizard-endpoint-configuration',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost@sha256:b517ba0196d2579c5393750335ed104da74b11ad2a6f0ae5da6907a7221ff2e0',
     'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost@sha256:b517ba0196d2579c5393750335ed104da74b11ad2a6f0ae5da6907a7221ff2e0',
     'ResolutionTime': datetime.datetime(2024, 5, 18, 18, 49, 23, 3000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 0,
   'CurrentServerlessConfig': {'MemorySizeInMB': 1024, 'MaxConcurrency': 10}}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2024, 5, 18, 18, 49, 22, 4230

# Invocation

In [24]:
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=b"40.7128,	-74.006, 0,	0, 0",
    ContentType="text/csv",
)

#print(response["Body"].read())
index_to_class[np.round(float(response["Body"].read().decode('utf-8')), 0)]

'RAINY'

In [26]:
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=b"-33.8688, 151.2093, 1, 0, 0",
    ContentType="text/csv",
)

#print(response["Body"].read())
index_to_class[np.round(float(response["Body"].read().decode('utf-8')), 0)]

'RAINY'

In [41]:
event = {
  "LATITUDE": "40.7128",
  "LONGITUDE": "-74.0060",
  "WEB_BROWSER": "1",
  "DEVICE": "2",
  "OPERATING_SYSTEM": "3"
}

In [42]:
dataset_df = json.dumps(event)
dataset_df = pd.json_normalize(json.loads(dataset_df))
dataset_df

,LATITUDE,LONGITUDE,WEB_BROWSER,DEVICE,OPERATING_SYSTEM
0,40.7128,-74.0060,1,2,3


In [65]:
arr = dataset_df.iloc[0].to_numpy()

In [66]:
formatted_str = ', '.join(arr)
formatted_str = formatted_str.replace(' ', '')  # Remove spaces

# Converting the formatted string to bytes
bytes_arr = bytes(formatted_str, 'utf-8')

In [67]:
bytes_arr

b'40.7128,-74.0060,1,2,3'

In [64]:
response = runtime.invoke_endpoint(EndpointName=endpoint_name,
                                    Body=bytes_arr,
                                    ContentType="text/csv")

#print(response["Body"].read())
index_to_class[np.round(float(response["Body"].read().decode('utf-8')), 0)]
    

'RAINY'

# Cleanup